<a href="https://colab.research.google.com/github/vedanshipathak/Mental-Health-Support-Chatbot/blob/main/Final_Model_mental_health_support.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain chromadb openai flask datasets evaluate torch transformers accelerate faiss-cpu


In [ ]:
import pandas as pd
import re
# Load the dataset into a pandas DataFrame
df = pd.read_csv(r'/content/counselchat-data.csv')

# Function to strip HTML tags (like <p>, <br>, etc.)
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', str(text))

# Apply cleaning to the 'answerText' column
df['clean_answer'] = df['answerText'].apply(remove_html_tags)

# Optional: Combine questionTitle and questionText for better context
df['full_question'] = df['questionTitle'].fillna('') + ' ' + df['questionText'].fillna('')

# Preview the cleaned data
df[['full_question', 'clean_answer']].head()
# Display first few rows
print(df.head())


In [ ]:
# Display column names
print("Column Names:", df.columns.tolist())


In [ ]:
import html

# Apply to the cleaned answer column
df['decoded_answer'] = df['clean_answer'].apply(html.unescape)

# Optional: also decode the question column if needed
df['decoded_question'] = df['full_question'].apply(html.unescape)

# Preview
df[['decoded_question', 'decoded_answer']].head()

In [ ]:
# Extract relevant columns
questions = df["decoded_question"].dropna().tolist()  # Remove NaN values if any
answers = df["decoded_answer"].dropna().tolist()

# Print a sample
print("Sample Question:", questions[0])
print("Sample Answer:", answers[0])

In [ ]:
!pip install langchain openai


In [ ]:
from langchain.prompts import PromptTemplate

# General Mental Health Support Prompt (already defined)
general_prompt = PromptTemplate(
    template="""
You are a compassionate therapist. Respond with empathy, support, and actionable advice.
Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

Client: {client_message}
Therapist:
""",
    input_variables=["client_message"],
)

# Anxiety-specific Prompt
anxiety_prompt = PromptTemplate(
    template="""
You are an empathetic therapist specialized in anxiety management. Provide calm, supportive, and actionable advice specifically targeted towards managing anxiety. Suggest practical techniques like mindfulness, breathing exercises, or grounding practices.

Client: {client_message}
Therapist:
""",
    input_variables=["client_message"],
)

# Depression-specific Prompt
depression_prompt = PromptTemplate(
    template="""
You are a compassionate therapist specialized in supporting clients experiencing sadness, depression, or feelings of hopelessness. Offer empathy, reassurance, and practical suggestions such as self-care activities, reaching out for support, or considering therapy.

Client: {client_message}
Therapist:
""",
    input_variables=["client_message"],
)

# Relationship-specific Prompt
relationship_prompt = PromptTemplate(
    template="""
You are a caring therapist specializing in relationship and family counseling. Provide empathetic and practical advice on communication strategies, understanding perspectives, conflict resolution, or improving interpersonal relationships.

Client: {client_message}
Therapist:
""",
    input_variables=["client_message"],
)


# Example usage:
client_message = "I've been feeling very anxious lately and don't know how to cope."
formatted_prompt = general_prompt.format(client_message=client_message)
print(formatted_prompt)


In [ ]:
def get_prompt(client_message):
    if any(word in client_message.lower() for word in ["anxious", "panic", "overwhelmed"]):
        return anxiety_prompt.format(client_message=client_message)
    elif any(word in client_message.lower() for word in ["depressed", "hopeless", "sad"]):
        return depression_prompt.format(client_message=client_message)
    elif any(word in client_message.lower() for word in ["relationship", "partner", "marriage", "family"]):
        return relationship_prompt.format(client_message=client_message)
    else:
        return general_prompt.format(client_message=client_message)

# Example Test
test_message = "I'm having serious conflicts with my wife, and I feel stuck."
selected_prompt = get_prompt(test_message)
print(selected_prompt)


In [ ]:
!pip install transformers torch accelerate


In [ ]:
from huggingface_hub import login
login()


In [ ]:
import requests

try:
    response = requests.get("https://huggingface.co", timeout=5)
    if response.status_code == 200:
        print(" Internet is working. Hugging Face is reachable!")
    else:
        print(" Internet issue: Received status code", response.status_code)
except requests.ConnectionError:
    print(" No internet connection detected!")


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

#  Load tokenizer
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

#  Fix: Set padding token to avoid ValueError when padding
tokenizer.pad_token = tokenizer.eos_token

#  Load model with half precision and auto device mapping
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

print("Model loaded successfully!")



In [ ]:
def generate_response(client_message):
    # Format the structured prompt
    prompt = f"""
    You are a compassionate therapist. Respond with empathy, support, and actionable advice.
    Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

    Client: {client_message}
    Therapist:
    """

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate response
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=200, temperature=0.7, top_p=0.9)

    # Decode and return the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

#  Test the chatbot
test_message = "I've been feeling very anxious lately and don't know how to cope."
response = generate_response(test_message)
print("\nChatbot Response:\n", response)


In [ ]:
!pip install datasets evaluate sacrebleu nltk


In [ ]:
!pip install rouge_score

In [ ]:
import evaluate
import math
import torch
import nltk
from transformers import AutoModelForCausalLM, AutoTokenizer

nltk.download("punkt")  # Needed for tokenization in BLEU & ROUGE

# Load BLEU and ROUGE metrics from Hugging Face's Evaluate library
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

# Function to calculate perplexity
def calculate_perplexity(response):
    inputs = tokenizer(response, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss
    perplexity = math.exp(loss.item())
    return perplexity


In [ ]:
def evaluate_response(generated_response, reference_response):
    # Compute BLEU score (expects full sentences, not tokenized words)
    bleu_score = bleu.compute(predictions=[generated_response], references=[[reference_response]])

    # Compute ROUGE score (expects full sentences)
    rouge_score = rouge.compute(predictions=[generated_response], references=[reference_response])

    # Compute perplexity
    perplexity = calculate_perplexity(generated_response)

    return {
        "BLEU Score": bleu_score["bleu"],
        "ROUGE Score": rouge_score["rouge1"],
        "Perplexity": perplexity
    }


In [ ]:
 import nltk
 nltk.download('punkt_tab')

In [ ]:
# Example chatbot response vs reference therapist response
generated_response = """
I'm sorry to hear that you're feeling anxious. It's completely normal to experience anxiety at times, but it's important to address it when it becomes overwhelming.
One helpful technique to manage anxiety is mindfulness meditation. This involves focusing on the present moment and accepting your thoughts and feelings without judgment.
"""
reference_response = """
I hear that you're feeling anxious, and that's completely understandable. A helpful way to manage anxiety is practicing deep breathing exercises or grounding techniques.
Taking a moment to focus on your breath or journaling can also be beneficial. Would you like to explore more coping strategies together?
"""

# Run evaluation
scores = evaluate_response(generated_response, reference_response)
print(scores)


In [ ]:
!pip install -U sentence-transformers



In [ ]:
!pip install faiss-cpu langchain chromadb


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load a free embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # Free alternative to OpenAIEmbeddings

# Example dataset
conversation_texts = [
    "How do I deal with anxiety?",
    "What should I do if I'm feeling very anxious?",
    "How can I manage stress effectively?",
    "I've been feeling overwhelmed at work, how do I handle it?"
]

# Convert questions to vector embeddings
vectors = embedding_model.encode(conversation_texts, convert_to_numpy=True)

# Create FAISS index
vector_array = np.array(vectors, dtype=np.float32)
faiss_index = faiss.IndexFlatL2(vector_array.shape[1])
faiss_index.add(vector_array)
print("FAISS Index Created Successfully with SentenceTransformers!")



In [ ]:
def retrieve_context(query, top_k=2):
    query_vector = np.array([embedding_model.encode(query)], dtype=np.float32)
    distances, indices = faiss_index.search(query_vector, k=top_k)

    # Fetch similar questions and associated answers explicitly
    similar_q_and_a = [
        (conversation_texts[i], df['decoded_answer'].iloc[i])
        for i in indices[0]
    ]
    return similar_q_and_a



In [ ]:

def generate_response_with_rag(client_message):
    # Retrieve similar past questions and answers
    context_qas = retrieve_context(client_message)

    # Format the retrieved context for clarity
    formatted_context = "\n\n".join([f"Q: {q}\nA: {a}" for q, a in context_qas])

    #  Enhanced prompt with clear instructions
    prompt = f"""
You are a compassionate therapist helping people with mental health concerns.

Below are similar past conversations that might help you understand the client's situation better and guide your response:

{formatted_context}

Based on the above context, now respond to the client below with empathy and actionable advice.
Your response should be emotionally supportive and, if appropriate, recommend mindfulness, self-care, or therapeutic strategies.

Client: {client_message}
Therapist:
"""

    # Tokenize and generate
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.6,
            top_p=0.9,
            repetition_penalty=1.2,
            eos_token_id=tokenizer.eos_token_id
        )

    # Decode and return just the therapist's message
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response.split("Therapist:")[-1].strip()
    return response


In [ ]:
test_messages = [
    "hello i am sad",
    "i feel overwhelmed by everything at work",
    "my mom and wife keep arguing and it’s stressing me out",
    "i can't sleep at night and keep overthinking"
]

for msg in test_messages:
    print("\nClient:", msg)
    print("Therapist:", generate_response_with_rag(msg))
    print(evaluate_response())

In [ ]:
evaluate_response

In [ ]:
# Example chatbot response vs reference therapist response
generated_response = """
I'm sorry to hear that you're feeling anxious. It's completely normal to experience anxiety at times, but it's important to address it when it becomes overwhelming.
One helpful technique to manage anxiety is mindfulness meditation. This involves focusing on the present moment and accepting your thoughts and feelings without judgment.
"""
reference_response = """
I hear that you're feeling anxious, and that's completely understandable. A helpful way to manage anxiety is practicing deep breathing exercises or grounding techniques.
Taking a moment to focus on your breath or journaling can also be beneficial. Would you like to explore more coping strategies together?
"""

# Run evaluation
scores = evaluate_response(generated_response, reference_response)
print(scores)


Backend Setup


In [ ]:

!pip install flask flask-cors --quiet
!pip install pyngrok --quiet
!pip install transformers accelerate --quiet


In [ ]:
from pyngrok import ngrok
# Set your authtoken here (paste your token inside the quotes)
ngrok.set_auth_token("NGROK_TOKEN")


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


general_prompt = PromptTemplate(
    template="""
You are a compassionate therapist. Respond with empathy, support, and actionable advice.
Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

Client: {client_message}
Therapist:""",
    input_variables=["client_message"],
)

anxiety_prompt = PromptTemplate(
    template="""
You are a calm and reassuring mental health assistant. The user is feeling anxious or overwhelmed.
Offer calming techniques, grounding exercises, and positive affirmations.

Client: {client_message}
Therapist:""",
    input_variables=["client_message"],
)

depression_prompt = PromptTemplate(
    template="""
You are a caring and understanding therapist. The user feels depressed, hopeless, or sad.
Listen supportively and offer encouragement, self-compassion practices, and gentle help.

Client: {client_message}
Therapist:""",
    input_variables=["client_message"],
)

relationship_prompt = PromptTemplate(
    template="""
You are a relationship counselor. The user is experiencing issues in relationships or family.
Help them understand their emotions and suggest communication strategies.

Client: {client_message}
Therapist:""",
    input_variables=["client_message"],
)

def get_prompt(client_message):
    message = client_message.lower()
    if any(word in message for word in ["anxious", "panic", "overwhelmed"]):
        return anxiety_prompt.format(client_message=client_message)
    elif any(word in message for word in ["depressed", "hopeless", "sad"]):
        return depression_prompt.format(client_message=client_message)
    elif any(word in message for word in ["relationship", "partner", "marriage", "family"]):
        return relationship_prompt.format(client_message=client_message)
    else:
        return general_prompt.format(client_message=client_message)

def run_model(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=150)
    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

app = Flask(__name__)
CORS(app)

@app.route("/chat", methods=["POST"])
def chat():
    data = request.json
    user_input = data.get("message", "")
    prompt = get_prompt(user_input)
    response = run_model(prompt)
    return jsonify({"response": response})

port = 5001
public_url = ngrok.connect(port)
print(f"🚀 Public URL: {public_url}")
app.run(port=port)
